# How does the number of star players on a team correlate with win percentage?

In [ ]:
sql_espn_diagnostic = '''
WITH team_name_map AS (
  SELECT * FROM (VALUES
    ('ATL', 'Atlanta Hawks'),
    ('BOS', 'Boston Celtics'),
    ('BRK', 'Brooklyn Nets'),
    ('CHA', 'Charlotte Hornets'),
    ('CHI', 'Chicago Bulls'),
    ('CLE', 'Cleveland Cavaliers'),
    ('DAL', 'Dallas Mavericks'),
    ('DEN', 'Denver Nuggets'),
    ('DET', 'Detroit Pistons'),
    ('GSW', 'Golden State Warriors'),
    ('HOU', 'Houston Rockets'),
    ('IND', 'Indiana Pacers'),
    ('LAC', 'Los Angeles Clippers'),
    ('LAL', 'Los Angeles Lakers'),
    ('MEM', 'Memphis Grizzlies'),
    ('MIA', 'Miami Heat'),
    ('MIL', 'Milwaukee Bucks'),
    ('MIN', 'Minnesota Timberwolves'),
    ('NOP', 'New Orleans Pelicans'),
    ('NYK', 'New York Knicks'),
    ('OKC', 'Oklahoma City Thunder'),
    ('ORL', 'Orlando Magic'),
    ('PHI', 'Philadelphia 76ers'),
    ('PHO', 'Phoenix Suns'),
    ('POR', 'Portland Trail Blazers'),
    ('SAC', 'Sacramento Kings'),
    ('SAS', 'San Antonio Spurs'),
    ('TOR', 'Toronto Raptors'),
    ('UTA', 'Utah Jazz'),
    ('WAS', 'Washington Wizards')
  ) AS t(abbrev, full_name)
),
star_players AS (
  SELECT
    ps.player,
    ps.team,
    ps.season
  FROM public.nba_api_player_stats ps
  WHERE ps.per >= 20
    AND ps.win_shares >= 5
    AND ps.box_plus_minus >= 2
),
star_counts AS (
  SELECT
    team,
    season,
    COUNT(*) AS num_star_players
  FROM star_players
  GROUP BY team, season
),
team_records AS (
  SELECT
    m.abbrev AS team,
    ws."Season" AS season,
    ws."Wins" AS wins,
    ws."Losses" AS losses,
    CAST(ws."Wins" AS FLOAT) / (ws."Wins" + ws."Losses") AS win_pct
  FROM public.web_scraped_standings ws
  JOIN team_name_map m
    ON ws."Team" = m.full_name
),
joined AS (
  SELECT
    tr.team,
    tr.season,
    COALESCE(sc.num_star_players, 0) AS num_star_players,
    tr.win_pct
  FROM team_records tr
  LEFT JOIN star_counts sc
    ON tr.team = sc.team AND tr.season = sc.season
)
SELECT
  num_star_players,
  ROUND(AVG(win_pct)::numeric, 3) AS avg_win_percentage,
  COUNT(*) AS num_teams
FROM joined
GROUP BY num_star_players
ORDER BY num_star_players;
'''

df_espn_diagnostic = pd.read_sql(sql_espn_diagnostic, con=engine)
pd.set_option('display.max_rows', None)
df_espn_diagnostic

# How many teams improved or declined in wins from 2022 to 2025?

In [ ]:
sql_espn_descriptive = '''
WITH team_wins AS (
  SELECT 
    "Team" AS team,
    "Season" AS season,
    "Wins" AS wins
  FROM public.web_scraped_standings
  WHERE "Season" IN (2022, 2025)
),
pivoted AS (
  SELECT
    team,
    MAX(CASE WHEN season = 2022 THEN wins END) AS wins_2022,
    MAX(CASE WHEN season = 2025 THEN wins END) AS wins_2025
  FROM team_wins
  GROUP BY team
)
SELECT
  team,
  wins_2022,
  wins_2025,
  (wins_2025 - wins_2022) AS win_change
FROM pivoted
ORDER BY win_change DESC;
'''

df_espn = pd.read_sql(sql_espn_descriptive, con=engine)
pd.set_option('display.max_rows', None)
df_espn
